#### <font color="#03997A"> Diplomado en Ciencia de datos UNAM </font>

##### <font color="#03997A"> Dr: Alejandro Pimentel  </font>

##### <font color="#03997A"> Modulo 8 Introducción al Deep Learning </font>

##### <font color="#03997A"> Ejercicio 4 CNN  Alumno: Ibarra Ramírez Sergio </font>

In [1]:
import tensorflow as tf
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import normalize
from sklearn.preprocessing import OneHotEncoder 

# Load data
X, y = fetch_openml('mnist_784', return_X_y=True)
X = normalize(X).reshape(-1, 28, 28, 1)

X_train, X_test = X[:60000], X[60000:] 
y_train, y_test = y[:60000], y[60000:]

y_train = OneHotEncoder().fit_transform(y_train.reshape(-1,1)).astype(float)
y_test = OneHotEncoder().transform(y_test.reshape(-1,1)).astype(float)

# Create dataset
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(10000).batch(100) 

# Initialize weights
b = tf.Variable(tf.zeros([10]))
W = tf.Variable(tf.random.truncated_normal([1568, 10], stddev=0.1))

conv1_f = tf.Variable(tf.random.truncated_normal([4, 4, 1, 32], stddev=0.1))
conv1_b = tf.Variable(tf.random.truncated_normal([32], stddev=0.1)) 

conv2_f = tf.Variable(tf.random.truncated_normal([4, 4, 32, 64], stddev=0.1))
conv2_b = tf.Variable(tf.random.truncated_normal([64], stddev=0.1))

# Training loop
optimizer = tf.optimizers.Adam()
epochs = 10

for epoch in range(epochs):

  for X_batch, y_batch in train_ds:

    with tf.GradientTape() as tape: 
    
      conv1 = tf.nn.conv2d(X_batch, conv1_f, strides=1, padding='SAME')  
      conv1 = tf.nn.bias_add(conv1, conv1_b)
      conv1 = tf.nn.relu(conv1)   

      conv2 = tf.nn.conv2d(conv1, conv2_f, strides=1, padding='SAME')
      conv2 = tf.nn.bias_add(conv2, conv2_b)
      conv2 = tf.nn.relu(conv2)
      
      # Flatten conv2 output preserving batch size
      conv2_shape = conv2.get_shape().as_list()
      flat = tf.reshape(conv2, [-1, conv2_shape[1] * conv2_shape[2] * conv2_shape[3]]) 

      dense = tf.matmul(flat, W) + b
      dense = tf.nn.relu(dense)

      loss = tf.losses.categorical_crossentropy(y_batch, dense)

    gradients = tape.gradient(loss, [W, b, conv1_f, conv1_b, conv2_f, conv2_b])
    optimizer.apply_gradients(zip(gradients, [W, b, conv1_f, conv1_b, conv2_f, conv2_b]))

  print(f'Epoch {epoch+1}, Loss: {loss.numpy():.4f}')

c:\Users\Sergio\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\datasets\_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


AttributeError: 'Series' object has no attribute 'reshape'